## This notebook shows how to use Orchestrator APIs for user experiments

In [ ]:
import os, traceback
from fabrictestbed.slice_manager import SliceManager, Status, SliceState
import json

In [ ]:
fabric_rc_location=os.environ['HOME']+"/work/fabric_config/fabric_rc"
if os.path.exists(fabric_rc_location):
    with open(fabric_rc_location, 'r') as f:
        for line in f:
            if line.startswith('export'):
                os.environ[line.split('=')[0].split('export')[1].strip()] = line.split('=')[1].strip()

In [ ]:
ssh_key_file_priv=os.environ["FABRIC_SLICE_PRIVATE_KEY_FILE"]
ssh_key_file_pub=os.environ["FABRIC_SLICE_PUBLIC_KEY_FILE"]

ssh_key_pub = None
with open (ssh_key_file_pub, "r") as myfile:
    ssh_key_pub=myfile.read()
    ssh_key_pub=ssh_key_pub.strip()

In [ ]:
print(os.environ["FABRIC_PROJECT_ID"])

## Create Slice Manager Object
Users can request tokens with different Project and Scopes by altering `project_name` and `scope` parameters in the refresh call below.

In [ ]:
slice_manager = SliceManager()

### Orchestrator API example to query for available resources

Force orchestrator to get list of available resources afresh from the broker by passing `force_refresh=True`. Otherwise, cached model is returned.

In [ ]:
status, advertised_topology = slice_manager.resources(force_refresh=True)

print(f"Status: {status}")
if status == Status.OK:
    print(f"Toplogy: {advertised_topology}")
else:
    print(f"Error: {advertised_topology}")

In [ ]:
if status == Status.OK:
    advertised_topology.draw()